<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/corpus_maker_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fabrique des Corpus ASAMLA
## Phase 2 : Synchronisation des Corpus ASAMLA
##### Last version : 06 juillet 2021
##### Author : gretiere

In [ ]:
!pip install gspread-formatting
# https://console.cloud.google.com/apis/credentials?project=mecenat-asamla-corpus
# client ID : 525819594891-bpjcuvkkrgbd5c6kf9jk2mn6t6b9kkoa.apps.googleusercontent.com
# client secret : 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
import pandas as pd
import numpy as np
import uuid
import random
import os
import re
import json
from gspread_formatting import *
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
!export PYTHONIOENCODING=utf8
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


### Ouverture du Corpus central (GC Drive compte Atos)
*   Cette feuille est la référence et est alimentée par les Corpus individuels


In [ ]:
# Ouverture de la Sheet Centrale (corpus_central_base)
sheet_central = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(sheet_central)
t_corpus = wb_central.get_worksheet(2)
data_t_corpus = t_corpus.get_all_values()
df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])
# on supprime les doublons
df_corpus_fr = df_corpus[['uid','expression','glossaire','état','date','commentaires','index']].drop_duplicates()

In [ ]:
df_corpus_fr

,uid,expression,glossaire,état,date,commentaires,index
0,f397cda1,A jeun,TRUE,validé,07/06/2021,,A
1,95c4a863,Accès fébrile concomitant chez un autre membre...,FALSE,supprimé,07/06/2021,,A
2,30579682,Accident,TRUE,validé,07/06/2021,,A
3,6f214e4c,Accident cérébral,TRUE,validé,07/06/2021,,A
4,8b195cf4,Allergie,TRUE,validé,07/06/2021,,A
...,...,...,...,...,...,...,...
167,fee1214e,Vitamines,TRUE,validé,07/06/2021,,V
168,39cba202,Vomissements,FALSE,validé,07/06/2021,,V
169,80aac977,Vos vaccinations sont-elles à jour ?,FALSE,validé,07/06/2021,,V
170,36e66161,Y a-t-il des cas contagieux à l'école ?,FALSE,validé,07/06/2021,,Y


### Traitement du corpus Français-ALL (toutes langues)
*   Une feuille = 1 Corpus
*   Feuille en partage (1 onglet = 1 langue) 

#### 2021 10 05 : Suppression de Peul, remplacé par Allemand



In [ ]:
# Corpus de travail des interprètes
#uri = "https://docs.google.com/spreadsheets/d/1-2SDHCFR5JOHCc4MiYSMo4V8yG7kfHbpIKV3q9ci6wE"
# Médical Vaccination
unique_interpret_sheet_uri = "https://docs.google.com/spreadsheets/d/1CclzYfFCW4srA3Lq_np2LpSrxj84JpcbzytL449DH8E"



In [ ]:
json_file = 'med_vac_synthese.json'
try:
  with open(json_file) as vk_dict:
      vk_lang_dict = json.load(vk_dict)
except BaseException as e:
  print(e)
  uri = unique_interpret_sheet_uri
  vk_lang_dict=[
  {'language':'Arabe','trigramme':'ams','uri':uri,'idx':1, 'update':'false', 'alphabet':"اآٱأإبتثجحخدذرزسشصضطظعغفقكلمنهةوؤيئىء"},
  {'language':'Anglais','trigramme':'eng','uri':uri,'idx':2, 'update':'false','alphabet':"AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz"},
  {'language':'Turc','trigramme':'tur','uri':uri,'idx':3, 'update':'false','alphabet':"AaÂâBbCcÇçDdEeFfGgĞğHhIıİiÎîJjKkLlMmNnOoÖöPpRrSsŞşTtUuÜüÛûVvYyZz"},
  {'language':'Russe','trigramme':'rus','uri':uri,'idx':4, 'update':'false','alphabet':"АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"},
  {'language':'Ukrainien','trigramme':'ukr','uri':uri,'idx':5, 'update':'false','alphabet':"АаБбВвГгҐґДдЕеЄєЖжЗзИиІіЇїЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЮюЯя"},
  {'language':'Roumain','trigramme':'rou','uri':uri,'idx':6, 'update':'false','alphabet':"AaĂăÂâBbCcDdEeFfGgHhIiÎîJjKkLlMmNnOoPpQqRrSsȘșTtȚțUuVvWwXxYyZz"},
  {'language':'Hongrois','trigramme':'hun','uri':uri,'idx':7, 'update':'false','alphabet':"Aa-Áá-Bb-Cc-Cscs-Dd-Ee-Éé-Ff-Gg-Gygy-Hh-Ii-Íí-Jj-Kk-Ll-Lyly-Mm-Nn-Oo-Óó-Öö-Őő-Pp-Qq-Rr-Ss-Szsz-Tt-Tyty-Uu-Úú-Üü-Űű-Vv-Ww-Xx-Yy-Zz-Zszs"},
  {'language':'Tigrinya','trigramme':'tig','uri':uri,'idx':8, 'update':'false','alphabet':"ሀሁሂሃሄህሆለሉሊላሌልሎሏሐሑሒሓሔሕሖሗመሙሚማሜምሞሟሠሡሢሣሤሥሦሧረሩሪራሬርሮሯሰሱሲሳሴስሶሷሸሹሺሻሼሽሾሿቀቁቂቃቄቅቆቇቐቑቒቓቔቕቖበቡቢባቤብቦቧቨቩቪቫቬቭቮቯተቱቲታቴትቶቷቸቹቺቻቼችቾቿኀኁኂኃኄኅኆኇነኑኒናኔንኖኗኘኙኚኛኜኝኞኟአኡኣኤእኦኧከኩኪካኬክኮኯኸኹኺኻኼኽኾወዉዊዋዌውዎዐዑዒዓዔዕዖዘዙዚዛዜዝዞዟዠዡዢዣዤዥዦዧየዩዪያዬይዮዯደዱዲዳዴድዶዷዸዹዺዻዼዽዾዿጀጁጂጃጄጅጆጇገጉጊጋጌግጎጏጘጙጚጛጜጝጞጟጠጡጢጣጤጥጦጧጨጩጪጫጬጭጮጯጰጱጲጳጴጵጶጷጸጹጺጻጼጽጾጿፀፁፂፃፄፅፆፇፈፉፊፋፌፍፎፏፐፑፒፓፔፕፖፗቈቊቋቌቍቘቚቛቜቝኈኊኋኌኍኰኲኳኴኵዀዂዃዄዅጐጒጓጔጕ"},
  {'language':'Albanais','trigramme':'alb','uri':uri,'idx':9, 'update':'false','alphabet':""},
  {'language':'Géorgien','trigramme':'geo','uri':uri,'idx':10, 'update':'false','alphabet':"ᲐᲑᲒᲓᲔᲕᲖᲗᲘᲙᲚᲛᲜᲝᲞᲟᲠᲡᲢᲣᲤᲥᲦᲧᲨᲩᲪᲫᲬᲭᲮᲯᲰ"},
  {'language':'Arménien','trigramme':'arm','uri':uri,'idx':11, 'update':'false','alphabet':"ԱաԲբԳգԴդԵեԶզԷէԸըԹթԺժԻիԼլԽխԾծԿկՀհՁձՂղՃճՄմՅյՆնՇշՈոՉչՊպՋջՌռՍսՎվՏտՐրՑցՒւՓփՔքՕօՖֆև"},
  {'language':'Dari','trigramme':'dar','uri':uri,'idx':12, 'update':'false','alphabet':"اآبپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی"},
  {'language':'Pashto','trigramme':'pst','uri':uri,'idx':13, 'update':'false','alphabet':""},
  {'language':'Fârsi','trigramme':'prs','uri':uri,'idx':14, 'update':'false','alphabet':""},
  {'language':'Azéri','trigramme':'aze','uri':uri,'idx':15, 'update':'false','alphabet':""},
  {'language':'Espagnol','trigramme':'esp','uri':uri,'idx':16, 'update':'false','alphabet':"AaÁáBbCcDdEeÉéFfGgHhIiÍíJjKkLlMmNnÑñOoÓóPpQqRrSsTtUuÚúÜüVvWwXxYyZz"},
  {'language':'Amharique','trigramme':'amh','uri':uri,'idx':17, 'update':'false','alphabet':""},
  {'language':'Allemand','trigramme':'all','uri':uri,'idx':18, 'update':'false','alphabet':"AaÄäBbCcDdEeFfGgHhIiJjKkLlMmNnOoÖöPpQqRrSsẞßTtUuÜüVvWwXxYyZz"}
  ]

In [ ]:
def fetch_corpus_trad(df_corpus, d_lang):
  # Chargement des corpus traduits
  uri = d_lang['uri']
  #ouverture de l'onglet Corpus dans un DF
  wb = gc.open_by_url(uri)
  t_corpus_trad = wb.get_worksheet(d_lang['idx'])
  data_t_corpus = t_corpus_trad.get_all_values()
  df_corpus_trad = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])
  # on vérifie que chaque expression possède la même clé
  df_corpus = pd.merge(df_corpus,df_corpus_trad[['uid','traduction']],on='uid', how='left')
  df_corpus['traduction'] = df_corpus['traduction'].apply(lambda x:x.strip())
  
  df_corpus.rename({'traduction': d_lang['trigramme']}, axis=1, inplace=True)
  return df_corpus

#### Chaque onglet est chargé dans un DF

In [ ]:
vk_df_corpus = []
for i in vk_lang_dict:
  i['uri'] = unique_interpret_sheet_uri
  if i['update']=='false':
    df_corpus = fetch_corpus_trad(df_corpus_fr,i)
    vk_df_corpus.append(df_corpus)



In [ ]:
def replace_unicode(word, vk_uni):
  for t_uni in vk_uni:
    word = word.replace (t_uni[1], t_uni[2])
  return word


In [ ]:
def narrow_no_break_space (s):
  s = " ".join(s.split())   # on vire tous les espaces en trop
  find = r'(\s*\?$)'        # on recherche le ? précédé ou pas d'espace \s*
  replace = u'\u202F'+ r'?'
  s = re.sub(find, replace, s)    # on remplace par un blanc insécable + le ?
  return s

### Application des bonnes règles typographiques pour la ponctuation française : espace fine insécable
    # https://www.typofute.com/l_espace_fine_insecable_dans_les_documents_html  
    # https://www.compart.com/en/unicode/U+202F









In [ ]:
# Application des bonnes règles typographiques pour la ponctuation française : espace fine insécable
df_corpus_fr['expression'] = df_corpus_fr['expression'].apply(lambda x:narrow_no_break_space(x))  
# itération langues
for i in vk_lang_dict:
  if i['update']=='false':
    df_corpus_fr = pd.merge(df_corpus_fr,vk_df_corpus[i['idx']-1][['uid',i['trigramme']]],on='uid', how='left')
    # Application des bonnes règles typographiques pour la ponctuation française : espace fine insécable
    df_corpus_fr[i['trigramme']] = df_corpus_fr[i['trigramme']].apply(lambda x:narrow_no_break_space(x))    
    if i['unicode_substition']!=[]:
      # substition de caractères unicodes (optionnel)
      print (i['trigramme'])
      df_corpus_fr[i['trigramme']] = df_corpus_fr[i['trigramme']].apply(lambda x:replace_unicode (x, i['unicode_substition']))    

ukr


In [ ]:
df_corpus_fr.loc[df_corpus_fr['uid']=='b6b565a5']['ukr']

83    Імунітет
Name: ukr, dtype: object

In [ ]:
ord(u' ')

8239

In [ ]:
df_corpus_fr.loc[df_corpus_fr['uid']=='e391fbcf']

,uid,expression,glossaire,état,date,commentaires,index,ams,eng,tur,rus,ukr,rou,hun,tig,alb,geo,arm,dar,pst,prs,aze,esp,amh,all
16,e391fbcf,Avez-vous déjà été opéré(e) ?,FALSE,validé,07/06/2021,,A,هل سبق أن خضعت لعملية جراحية؟,Have you ever had an operation ?,Daha önce ameliyat oldunuzmu ?,Вам когда-нибудь делали операцию ?,Вам коли-небудь робили операцію ?,Ați fost deja operat ?,Volt műtve ?,መጥባሕቲ ገርካዶ ትፈልጥ ?,A keni bërë ndonjëherë një operacion ?,ოპერაცია თუ გაქვთ როდესმე გაკეთებული ?,Դուք վիրահատված եղե՞լ եք,آیا تا به حال عمل کرده اید؟,,,,¿Ya fue operado ?,,


In [ ]:
# vérification de la présence de la double traduction en hongrois
df_corpus_fr.loc[df_corpus_fr['uid']=='f80074ba']

,uid,expression,glossaire,état,date,commentaires,index,ams,eng,tur,rus,ukr,rou,hun,tig,alb,geo,arm,dar,pst,prs,aze,esp,amh,all
100,f80074ba,Méningite,TRUE,validé,07/06/2021,,M,التهاب السحايا,Meningitis,Menenjit,Менингит,Менінгіт,Meningită,Meningitisz (agyhártyagyulladás),ናይ ዓጽሚ ምግታር ሕማም,Meningjiti,მენინგიტი,մենինգիտ,,,,,Meningitis,,
101,f80074ba,Méningite,TRUE,validé,07/06/2021,,M,التهاب السحايا,Meningitis,Menenjit,Менингит,Менінгіт,Meningită,Agyhártyagyulladás,ናይ ዓጽሚ ምግታር ሕማም,Meningjiti,მენინგიტი,մենինգիտ,,,,,Meningitis,,


**Sauvegarde de la synthèse**

In [ ]:
def format_feuille(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)
  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:G', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=7)
  set_column_width(ws, 'A', 100)
  set_column_width(ws, 'B', 300)

In [ ]:
df_corpus_synth = df_corpus_fr.copy()
nom_onglet = 'med_vac_synthese'
# si l'onglet existe déjà
try:
  ws = wb_central.worksheet(nom_onglet)
  wb_central.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb_central.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb_central.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_corpus_synth)
format_feuille(wb_central, nom_onglet)

# FIN

In [ ]:
df_corpus_synth

In [ ]:
vk_lang_dict

In [ ]:
unicode_substition = [("A", u"\u0041", u"\u0410"),("B", u"\u0042", u"\u0412"),("E", u"\u0045", u"\u0415"),("I", u"\u0049", u"\u0406"), ("K", u"\u004B", u"\u041A"),("M", u"\u004D", u"\u041C"),("P", u"\u0050", u"\u0420"),("T",u"\u0054", u"\u0422")]

In [ ]:
ord(u'Р')

1056

In [ ]:
vk_lang_dict[4]["unicode_substition"]=[("A", u"\u0041", u"\u0410"),("B", u"\u0042", u"\u0412"),("E", u"\u0045", u"\u0415"),("I", u"\u0049", u"\u0406"), ("K", u"\u004B", u"\u041A"),("M", u"\u004D", u"\u041C"),("P", u"\u0050", u"\u0420"),("T",u"\u0054", u"\u0422")]

In [ ]:
unicode_substition = vk_lang_dict[4]["unicode_substition"]

In [ ]:
unicode_substition

[('A', 'A', 'А'),
 ('B', 'B', 'В'),
 ('E', 'E', 'Е'),
 ('I', 'I', 'І'),
 ('K', 'K', 'К'),
 ('M', 'M', 'М'),
 ('P', 'P', 'Р'),
 ('T', 'T', 'Т')]

In [ ]:
ord(u'І')

1030

In [ ]:
# sauvegarde du fichier de paramétrage
with open('med_vac_synthese.json', 'w') as fp:
    json.dump(vk_lang_dict, fp)

In [ ]:
df_corpus_synth.loc[df_corpus_synth['uid']=='b6b565a5']['ukr']


83    Імунітет
Name: ukr, dtype: object

In [ ]:
s = "Question?"

In [ ]:
df_corpus_synth['ukr'] = df_corpus_synth['ukr'].apply(lambda x:replace_unicode (x, unicode_substition))

In [ ]:
narrow_no_break_space(s)

'Question\u202f?'

In [ ]:
ord(u"\u202F")

8239